# Building Agents

시작하기 전에 [이 슬라이드](https://docs.google.com/presentation/d/13c0L1CQWAL7fuCXakOqjkvoodfynPJI4Hw_4H76okVU/edit?usp=sharing)를 참고하여 배경 지식을 확인하세요!

이메일 어시스턴트를 처음부터 구축해 보겠습니다. 
1) 랭그래프로 에이전트 아키텍처 설계
2) 랭스미스로 테스트
3) 휴먼 인 더 루프(Human-in-the-loop), 
4) 메모리 순으로 진행합니다. 

이 다이어그램은 각 구성 요소가 어떻게 결합되는지 보여줍니다:

![overview-img](assets/overview.png)

## Resources

- Notebook Reference: [agent.ipynb](https://github.com/langchain-ai/agents-from-scratch/blob/main/notebooks/agent.ipynb)
- For LangSmith Studio: [src/email_assistant](https://github.com/langchain-ai/agents-from-scratch/tree/main/src/email_assistant)
- Slides: [Building Ambient Agents with LangGraph - Building Agents & Evaluations.pdf](https://files.cdn.thinkific.com/file_uploads/967498/attachments/5f6/a6b/958/Building_Ambient_Agents_with_LangGraph_-_Building_Agents___Evaluations.pdf)



## 환경 설정

In [ ]:
from dotenv import load_dotenv


load_dotenv("../../.env", override=True)